# Parte 1

Implante um servidor web em um cluster KUBERNETES com auto-escalamento horizontal automático.

Pede- se duas implantações:
- Cluster Kubernetes usando minikube
- Cluster AWS EKS

Entregas
- Roteiros de implantação e teste
- Códigos comentados
- Telas
- Artigo formato SBC ou IEEE
- Repositório GITHUB

Apresente um problema e discuta criticamente as implantações. Exemplo de título: técnicas de auto-escalonamento. Procure teses sobre load balancing. Na introdução, diga onde está sua contribuição. Máximo de 12 páginas.

## Horizontal Pod Autoscaler - Minikube



https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/

A HorizontalPodAutoscaler (HPA) automatically updates a workload resource (such as a Deployment or StatefulSet), with the aim of automatically scaling the workload to match demand.

Both **Deployment** and **StatefulSet** are types of workload resources used to manage the lifecycle of applications. Stateless applications do not retain any client data (or "state") between requests. Each request from a client is treated as an independent transaction that is unrelated to any previous request. Stateful applications maintain state across client interactions; this means that they remember previous interactions and use this information to influence future interactions.

**Deployment:** Suitable for stateless applications like web servers, APIs, or other services where any instance of the application can handle incoming requests.
**StatefulSet:** Suitable for databases, distributed systems like Kafka or Cassandra, or any application where each instance needs to maintain its state across restarts.

Horizontal scaling means that the response to increased load is to deploy more Pods. Vertical scaling means assigning more resources (for example: memory or CPU) to the Pods that are already running for the workload.

If the load decreases, and the number of Pods is above the configured minimum, the HorizontalPodAutoscaler instructs the workload resource (the Deployment, StatefulSet, or other similar resource) to scale back down.

Apache HTTP Server, commonly referred to as Apache httpd or simply Apache, is an open-source web server software. Apache is widely used by web hosting providers to serve websites and web applications. Apache listens for incoming HTTP requests on specified ports (usually port 80 for HTTP and port 443 for HTTPS) and processes these requests to serve the requested web pages or resources.

Before you begin, you need to have a Kubernetes cluster, and the kubectl command-line tool must be configured to communicate with your cluster. It is recommended to run this tutorial on a cluster with at least two nodes that are not acting as control plane hosts. Your Kubernetes server must be at or later than version 1.23. You also need to use a cluster that has a Metrics Server deployed and configured. 

Segui o tutorial de instalação do minikube no meu PC local.

```sh
alex@alex-inspiron:~$ minikube status
minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured

alex@alex-inspiron:~$ kubectl get nodes
NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   19m   v1.30.0

alex@alex-inspiron:~$ minikube stop
✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 node stopped.

```

```sh
minikube start --nodes=2

❗  You cannot change the number of nodes for an existing minikube cluster. Please use 'minikube node add' to add nodes to an existing cluster.
```
Metrics server is enabled (I enabled it right after installing), so I don't need to enable it again.

```sh
alex@alex-inspiron:~$ kubectl get nodes
NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   27m   v1.30.0
alex@alex-inspiron:~$ minikube node add
😄  Adding node m02 to cluster minikube as [worker]
❗  Cluster was created without any CNI, adding a node to it might cause broken networking.
👍  Starting "minikube-m02" worker node in "minikube" cluster
🚜  Pulling base image v0.0.44 ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.30.0 on Docker 26.1.1 ...
🔎  Verifying Kubernetes components...
🏄  Successfully added m02 to minikube!
alex@alex-inspiron:~$ kubectl get nodes
NAME           STATUS   ROLES           AGE   VERSION
minikube       Ready    control-plane   28m   v1.30.0
minikube-m02   Ready    <none>          13s   v1.30.0
```

You now have a multi-node setup, with `minikube` as the control-plane node and `minikube-m02` as a worker node. In Kubernetes, when you create a node using Minikube, it may not automatically assign the role as `worker` for nodes added to the cluster. Instead, they appear with the role `<none>`. This is typical behavior in Minikube, as it doesn't automatically label nodes with specific roles like `worker` or `control-plane`.

Adicionei mais um node:

```sh
alex@alex-inspiron:~$ minikube node add
😄  Adding node m03 to cluster minikube as [worker]
👍  Starting "minikube-m03" worker node in "minikube" cluster
🚜  Pulling base image v0.0.44 ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.30.0 on Docker 26.1.1 ...
🔎  Verifying Kubernetes components...
🏄  Successfully added m03 to minikube!
alex@alex-inspiron:~$ kubectl get nodes
NAME           STATUS   ROLES           AGE     VERSION
minikube       Ready    control-plane   34m     v1.30.0
minikube-m02   Ready    <none>          6m25s   v1.30.0
minikube-m03   Ready    <none>          6s      v1.30.0

alex@alex-inspiron:~$ kubectl label node minikube-m02 node-role.kubernetes.io/worker=worker
node/minikube-m02 labeled
alex@alex-inspiron:~$ kubectl label node minikube-m03 node-role.kubernetes.io/worker=worker
node/minikube-m03 labeled
```

```sh
alex@alex-inspiron:~$ kubectl config current-context
minikube
```

This shows that `kubectl` is set to interact with your Minikube cluster.

```sh
alex@alex-inspiron:~$ kubectl cluster-info
Kubernetes control plane is running at https://192.168.49.2:8443
CoreDNS is running at https://192.168.49.2:8443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
```




### Run and expose php-apache server

Start a Deployment that runs a container using the `hpa-example` image, and expose it as a Service using the following manifest:

application/php-apache.yaml
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: php-apache
spec:
  selector:
    matchLabels:
      run: php-apache
  template:
    metadata:
      labels:
        run: php-apache
    spec:
      containers:
      - name: php-apache
        image: registry.k8s.io/hpa-example
        ports:
        - containerPort: 80
        resources:
          limits:
            cpu: 500m
          requests:
            cpu: 200m
---
apiVersion: v1
kind: Service
metadata:
  name: php-apache
  labels:
    run: php-apache
spec:
  ports:
  - port: 80
  selector:
    run: php-apache

```

This file describes a Kubernetes Deployment and a Service for running a `php-apache` application. This file contains two main sections: the Deployment and the Service.

A Deployment manages a set of replicated Pods for a given application. In this case, it's used to deploy a PHP application running on an Apache server.

- **apiVersion: apps/v1:** Specifies the API version for the Deployment resource.

- **kind: Deployment:** Indicates that this configuration is for a Deployment.

- **metadata:**
  - **name: php-apache:** The name of the Deployment, which will be used to identify it in the cluster.

- **spec:** Defines the desired state and behavior of the Deployment.
  - **selector:** Specifies how to identify the Pods managed by this Deployment.
    - **matchLabels:** The Deployment will manage Pods with the label `run: php-apache`.
  
  - **template:** Defines the Pod template used by the Deployment to create Pods.
    - **metadata:**
      - **labels:** Adds the label `run: php-apache` to the Pods created by this template.
    
    - **spec:**
      - **containers:** Specifies the container settings for the Pods.
        - **name: php-apache:** The name of the container.
        - **image: registry.k8s.io/hpa-example:** The container image to be used. This example uses an image suitable for demonstrating Horizontal Pod Autoscaling (HPA).
        - **ports:** Defines the network ports exposed by the container.
          - **containerPort: 80:** Exposes port 80, which is typically used for HTTP traffic.
        
        - **resources:** Specifies resource requests and limits for the container.
          - **limits:**
            - **cpu: 500m:** Sets a CPU limit of 500 milliCPU (half a CPU core) for the container.
          - **requests:**
            - **cpu: 200m:** Requests 200 milliCPU, indicating the amount of CPU guaranteed to the container.

A Service in Kubernetes is an abstraction that defines a logical set of Pods and a policy by which to access them, usually by a stable network endpoint.

- **apiVersion: v1:** Specifies the API version for the Service resource.

- **kind: Service:** Indicates that this configuration is for a Service.

- **metadata:**
  - **name: php-apache:** The name of the Service.
  - **labels:** The label `run: php-apache` helps identify this Service.

- **spec:**
  - **ports:**
    - **port: 80:** Exposes the Service on port 80.
  
  - **selector:** Defines which Pods this Service targets by matching the label `run: php-apache`.

The Deployment ensures that the specified number of Pods (replicas) are running. Each Pod runs a container using the specified image (`registry.k8s.io/hpa-example`) with the defined resource constraints. It exposes port 80 for HTTP traffic.

The Service provides a stable endpoint to access the Pods managed by the Deployment. It uses a label selector to route traffic to Pods with the `run: php-apache` label, effectively load balancing the requests among them.

```sh
kubectl apply -f https://k8s.io/examples/application/php-apache.yaml
```
*deployment.apps/php-apache created*  
*service/php-apache created*


Há um problema:

```sh
alex@alex-inspiron:~/Documents/Projects/CloudComputing/HPA_minikube$ kubectl get pods --all-namespaces
NAMESPACE              NAME                                        READY   STATUS             RESTARTS       AGE
default                php-apache-678865dd57-244tq                 0/1     CrashLoopBackOff   6 (2m1s ago)   6m21s
...
```

The `php-apache` pod is in a `CrashLoopBackOff` state, indicating that it is repeatedly failing to start. 

```sh
alex@alex-inspiron:~/Documents/Projects/CloudComputing/HPA_minikube$ kubectl logs php-apache-678865dd57-244tq
AH00557: apache2: apr_sockaddr_info_get() failed for php-apache-678865dd57-244tq
AH00558: apache2: Could not reliably determine the server's fully qualified domain name, using 127.0.0.1. Set the 'ServerName' directive globally to suppress this message
AH00557: apache2: apr_sockaddr_info_get() failed for php-apache-678865dd57-244tq
AH00558: apache2: Could not reliably determine the server's fully qualified domain name, using 127.0.0.1. Set the 'ServerName' directive globally to suppress this message
[Sun Aug 04 04:42:37.378877 2024] [mpm_prefork:notice] [pid 1] AH00163: Apache/2.4.10 (Debian) PHP/5.6.14 configured -- resuming normal operations
[Sun Aug 04 04:42:37.378915 2024] [core:notice] [pid 1] AH00094: Command line: 'apache2 -D FOREGROUND'
[Sun Aug 04 04:42:37.801783 2024] [mpm_prefork:notice] [pid 1] AH00169: caught SIGTERM, shutting down
```

**apr_sockaddr_info_get() failed for php-apache-678865dd57-244tq**:
This error suggests that Apache is having trouble resolving the hostname `php-apache-678865dd57-244tq`. This can occur if the hostname cannot be resolved to an IP address.

**Could not reliably determine the server's fully qualified domain name, using 127.0.0.1**:
This message indicates that Apache is unable to determine the server's fully qualified domain name (FQDN) and is defaulting to `127.0.0.1`. It's a common warning when the `ServerName` directive is not set in the Apache configuration.

**caught SIGTERM, shutting down**:
This indicates that the Apache server received a termination signal and is shutting down. This could be related to the container crashing or being killed by Kubernetes.

Dei `minikube stop´ seguido de ´minikube start` e está running.

### Create the HorizontalPodAutoscaler

The `kubectl autoscale` command is used to create an HPA, which automatically adjusts the number of pods in a deployment based on observed CPU utilization or other select metrics.

```sh
kubectl autoscale deployment php-apache --cpu-percent=50 --min=1 --max=10
```
*horizontalpodautoscaler.autoscaling/php-apache autoscaled*

This command reates a HorizontalPodAutoscaler that maintains between 1 and 10 replicas of the Pods controlled by the php-apache Deployment that you created in the first step of these instructions.

Roughly speaking, the HPA controller will increase and decrease the number of replicas (by updating the Deployment) to maintain an average CPU utilization across all Pods of 50%. The Deployment then updates the ReplicaSet - this is part of how all Deployments work in Kubernetes - and then the ReplicaSet either adds or removes Pods based on the change to its `.spec`.

Since each pod requests 200 milli-cores by `kubectl run`, this means an average CPU usage of 100 milli-cores. 

You can check the current status of the newly-made HorizontalPodAutoscaler, by running:

```sh
alex@alex-inspiron:~/Documents/Projects/CloudComputing/HPA_minikube$ kubectl get hpa
NAME         REFERENCE               TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
php-apache   Deployment/php-apache   cpu: <unknown>/50%   1         10        1          37s
```

The current CPU consumption should 0% as there are no clients sending requests to the server (the TARGET column shows the average across all the Pods controlled by the corresponding deployment).

### Increase the Load

You'll start a different Pod to act as a client. The container within the client Pod runs in an infinite loop, sending queries to the php-apache service.

```sh
# Run this in a separate terminal so that the load generation continues and you can carry on with the rest of the steps
kubectl run -i --tty load-generator --rm --image=busybox:1.28 --restart=Never -- /bin/sh -c "while sleep 0.01; do wget -q -O- http://php-apache; done"
```

This command creates a temporary pod named `load-generator` using the `busybox:1.28` image. It runs a shell script in the pod that repeatedly sends HTTP requests to the `php-apache` service at a very high frequency (every 0.01 seconds). This effectively generates load or traffic to test the performance and scaling behavior of the `php-apache` deployment.

Ao rodar o comando acima, é retornado:

*wget: bad address 'php-apache'*

It indicates that the `load-generator` pod is unable to resolve the hostname `php-apache`. This is typically due to DNS issues or incorrect service configuration in your Kubernetes cluster.

```sh
alex@alex-inspiron:~$ kubectl describe service php-apache
Name:              php-apache
Namespace:         default
Labels:            run=php-apache
Annotations:       <none>
Selector:          run=php-apache
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.106.3.165
IPs:               10.106.3.165
Port:              <unset>  80/TCP
TargetPort:        80/TCP
Endpoints:         <none>
```

```sh
alex@alex-inspiron:~$ kubectl get pods --show-labels
NAME                          READY   STATUS             RESTARTS         AGE    LABELS
php-apache-678865dd57-244tq   0/1     CrashLoopBackOff   31 (2m25s ago)   100m   pod-template-hash=678865dd57,run=php-apache
```

The issue is that the `php-apache` service has no endpoints, meaning there are no pods currently backing the service. This is why the `wget: bad address 'php-apache'` error occurs and why `nslookup` can't resolve the service name. The reason there are no endpoints for the `php-apache` service is likely due to the pod being in a `CrashLoopBackOff` state. This state indicates that the pod is repeatedly failing to start.

Vou deletar o cluster e refazê-lo com 2 nodes:

```sh
alex@alex-inspiron:~$ minikube delete
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Deleting container "minikube-m02" ...
🔥  Deleting container "minikube-m03" ...
🔥  Removing /home/alex/.minikube/machines/minikube ...
🔥  Removing /home/alex/.minikube/machines/minikube-m02 ...
🔥  Removing /home/alex/.minikube/machines/minikube-m03 ...
💀  Removed all traces of the "minikube" cluster.
alex@alex-inspiron:~$ rm -rf ~/.minikube
alex@alex-inspiron:~$ docker system prune -f
Deleted Containers:
f1a6de1d2168c16100313d6e1aefab04f4bfdb8cc4f3aee69626a31cc4d381a2
737b16c1f5254ffcde2420fc651482469182007c691b967c81c8595b2868c984
Total reclaimed space: 0B
```

```sh
alex@alex-inspiron:~$ minikube start --nodes=2
...
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
```

Refiz os passos anteriores após diminuir para 2 nós.

```sh
alex@alex-inspiron:~$ kubectl run -i --tty load-generator --rm --image=busybox:1.28 --restart=Never -- /bin/sh -c "while sleep 0.01; do wget -q -O- http://php-apache; done"
If you don't see a command prompt, try pressing enter.
OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!OK!
```

```sh
# No terminal que não tá rodando o run load-generator
kubectl get hpa php-apache --watch
```

```sh
alex@alex-inspiron:~$ kubectl get hpa php-apache --watch
NAME         REFERENCE               TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
php-apache   Deployment/php-apache   cpu: <unknown>/50%   1         10        1          8h
```

Tem algo errado, não tá constando o resultado esperado. Descobri o problema: o metrics-server não tava habilitado. Agora funcionou:

```sh
alex@alex-inspiron:~$ kubectl get hpa php-apache --watch
NAME         REFERENCE               TARGETS       MINPODS   MAXPODS   REPLICAS   AGE
php-apache   Deployment/php-apache   cpu: 0%/50%   1         10        1          8h
php-apache   Deployment/php-apache   cpu: 20%/50%   1         10        1          8h
php-apache   Deployment/php-apache   cpu: 250%/50%   1         10        1          8h
php-apache   Deployment/php-apache   cpu: 250%/50%   1         10        4          8h
php-apache   Deployment/php-apache   cpu: 250%/50%   1         10        5          8h
php-apache   Deployment/php-apache   cpu: 147%/50%   1         10        5          8h
php-apache   Deployment/php-apache   cpu: 68%/50%    1         10        5          8h
php-apache   Deployment/php-apache   cpu: 55%/50%    1         10        5          8h
php-apache   Deployment/php-apache   cpu: 55%/50%    1         10        6          8h
php-apache   Deployment/php-apache   cpu: 46%/50%    1         10        6          8h
php-apache   Deployment/php-apache   cpu: 45%/50%    1         10        6          8h
php-apache   Deployment/php-apache   cpu: 46%/50%    1         10        6          8h
php-apache   Deployment/php-apache   cpu: 46%/50%    1         10        6          8h
```

Here, CPU consumption has increased to 250% of the request. As a result, the Deployment was resized to 6 replicas.

Rodei de novo esses comandos, obtendo:

```sh
alex@alex-inspiron:~$ kubectl get hpa php-apache --watch
NAME         REFERENCE               TARGETS       MINPODS   MAXPODS   REPLICAS   AGE
php-apache   Deployment/php-apache   cpu: 0%/50%   1         10        1          11h
php-apache   Deployment/php-apache   cpu: 117%/50%   1         10        1          11h
php-apache   Deployment/php-apache   cpu: 117%/50%   1         10        3          11h
php-apache   Deployment/php-apache   cpu: 140%/50%   1         10        3          11h
^C
alex@alex-inspiron:~kubectl get deployment php-apachehe
NAME         READY   UP-TO-DATE   AVAILABLE   AGE
php-apache   5/6     6            5           11h
```

It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled in any way it may happen that the final number of replicas will differ from this example.

### Stop generating load

Stop sending the load: in the terminal where you created the Pod that runs a busybox image, terminate the load generation by typing `<Ctrl> + C`.

Then verify the result state (after a minute or so):

```sh
alex@alex-inspiron:~$ kubectl get hpa php-apache --watch
NAME         REFERENCE               TARGETS       MINPODS   MAXPODS   REPLICAS   AGE
php-apache   Deployment/php-apache   cpu: 0%/50%   1         10        3          11h
php-apache   Deployment/php-apache   cpu: 0%/50%   1         10        3          11h
php-apache   Deployment/php-apache   cpu: 0%/50%   1         10        1          11h
^C
alex@alex-inspiron:~$ kubectl get deployment php-apache
NAME         READY   UP-TO-DATE   AVAILABLE   AGE
php-apache   1/1     1            1           11h
```
Once CPU utilization dropped to 0, the HPA automatically scaled the number of replicas back down to 1. Autoscaling the replicas may take a few minutes.


### Autoscaling on multiple metrics and custom metrics

You can introduce additional metrics to use when autoscaling the `php-apache` Deployment by making use of the `autoscaling/v2` API version.

First, get the YAML of your HorizontalPodAutoscaler in the autoscaling/v2 form:

```sh
kubectl get hpa php-apache -o yaml > /tmp/hpa-v2.yaml
```

The targetCPUUtilizationPercentage field has been replaced with an array called metrics.

```yaml
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 50
```

The CPU utilization metric is a resource metric, since it is represented as a percentage of a resource specified on pod containers.

You can specify other resource metrics besides CPU. By default, the only other supported resource metric is `memory`. These resources do not change names from cluster to cluster, and should always be available, as long as the `metrics.k8s.io` API is available.

You can also specify resource metrics in terms of direct values, instead of as percentages of the requested value, by using a `target.type` of AverageValue instead of Utilization, and setting the corresponding `target.averageValue` field instead of the `target.averageUtilization`.

```yaml
  metrics:
  - type: Resource
    resource:
      name: memory
      target:
        type: AverageValue
        averageValue: 500Mi
```

There are two other types of metrics, both of which are considered custom metrics: pod metrics and object metrics. These metrics may have names which are cluster specific, and require a more advanced cluster monitoring setup.

The first of these alternative metric types is `pod metrics`. These metrics describe Pods, and are averaged together across Pods and compared with a target value to determine the replica count. They work much like resource metrics, except that they only support a target type of `AverageValue`.

```yaml
type: Pods
pods:
  metric:
    name: packets-per-second
  target:
    type: AverageValue
    averageValue: 1k
```

The second alternative metric type is `object metrics`. These metrics describe a different object in the same namespace, instead of describing Pods. The metrics are not necessarily fetched from the object; they only describe it. Object metrics support target types of both `Value` and `AverageValue`. 

With `Value`, the target is compared directly to the returned metric from the API. With `AverageValue`, the value returned from the custom metrics API is divided by the number of Pods before being compared to the target. 

The following example is the YAML representation of the `requests-per-second` metric.

```yaml
type: Object
object:
  metric:
    name: requests-per-second
  describedObject:
    apiVersion: networking.k8s.io/v1
    kind: Ingress
    name: main-route
  target:
    type: Value
    value: 2k
```

If you provide multiple such metric blocks, the HorizontalPodAutoscaler will consider each metric in turn. The HorizontalPodAutoscaler will calculate proposed replica counts for each metric, and then choose the one with the highest replica count.

For example, if you had your monitoring system collecting metrics about network traffic, you could update the definition above using kubectl edit to look like this:

```yaml
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: php-apache
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: php-apache
  minReplicas: 1
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 50
  - type: Pods
    pods:
      metric:
        name: packets-per-second
      target:
        type: AverageValue
        averageValue: 1k
  - type: Object
    object:
      metric:
        name: requests-per-second
      describedObject:
        apiVersion: networking.k8s.io/v1
        kind: Ingress
        name: main-route
      target:
        type: Value
        value: 10k
status:
  observedGeneration: 1
  lastScaleTime: <some-time>
  currentReplicas: 1
  desiredReplicas: 1
  currentMetrics:
  - type: Resource
    resource:
      name: cpu
    current:
      averageUtilization: 0
      averageValue: 0
  - type: Object
    object:
      metric:
        name: requests-per-second
      describedObject:
        apiVersion: networking.k8s.io/v1
        kind: Ingress
        name: main-route
      current:
        value: 10k
```

Then, your HorizontalPodAutoscaler would attempt to ensure that each pod was consuming roughly 50% of its requested CPU, serving 1000 packets per second, and that all pods behind the main-route Ingress were serving a total of 10000 requests per second.

### Autoscaling on more specific metrics

Many metrics pipelines allow you to describe metrics either by name or by a set of additional descriptors called `labels`. For all non-resource metric types (pod, object, and external, described below), you can specify an additional label selector which is passed to your metric pipeline.

For instance, if you collect a metric `http_requests` with the `verb` label, you can specify the following metric block to scale only on GET requests:

```yaml
type: Object
object:
  metric:
    name: http_requests
    selector: {matchLabels: {verb: GET}}
```

This selector uses the same syntax as the full Kubernetes label selectors. The monitoring pipeline determines how to collapse multiple series into a single value, if the name and selector match multiple series. The selector is additive, and cannot select metrics that describe objects that are not the target object (the target pods in the case of the `Pods` type, and the described object in the case of the `Object` type).

### Autoscaling on metrics not related to Kubernetes objects

Applications running on Kubernetes may need to autoscale based on metrics that don't have an obvious relationship to any object in the Kubernetes cluster, such as metrics describing a hosted service with no direct correlation to Kubernetes namespaces. In Kubernetes 1.10 and later, you can address this use case with external metrics.

Using external metrics requires knowledge of your monitoring system; the setup is similar to that required when using custom metrics. External metrics allow you to autoscale your cluster based on any metric available in your monitoring system. Provide a `metric` block with a `name` and `selector`, as above, and use the `External` metric type instead of `Object`.

If multiple time series are matched by the `metricSelector`, the sum of their values is used by the HorizontalPodAutoscaler. External metrics support both the `Value` and `AverageValue` target types, which function exactly the same as when you use the `Object` type.

For example if your application processes tasks from a hosted queue service, you could add the following section to your HorizontalPodAutoscaler manifest to specify that you need one worker per 30 outstanding tasks.

```yaml
- type: External
  external:
    metric:
      name: queue_messages_ready
      selector:
        matchLabels:
          queue: "worker_tasks"
    target:
      type: AverageValue
      averageValue: 30
```

When possible, it's preferable to use the custom metric target types instead of external metrics, since it's easier for cluster administrators to secure the custom metrics API. The external metrics API potentially allows access to any metric, so cluster administrators should take care when exposing it.

### Appendix: Horizontal Pod Autoscaler Status Conditions

When using the `autoscaling/v2 ` form of the HorizontalPodAutoscaler, you will be able to see status conditions set by Kubernetes on the HorizontalPodAutoscaler. These status conditions indicate whether or not the HorizontalPodAutoscaler is able to scale, and whether or not it is currently restricted in any way.

The conditions appear in the `status.conditions` field. To see the conditions affecting a HorizontalPodAutoscaler, we can use `kubectl describe hpa`:

```sh
alex@alex-inspiron:~$ kubectl describe hpa php-apache
Name:                                                  php-apache
Namespace:                                             default
Labels:                                                <none>
Annotations:                                           <none>
CreationTimestamp:                                     Sun, 04 Aug 2024 03:48:49 -0300
Reference:                                             Deployment/php-apache
Metrics:                                               ( current / target )
  resource cpu on pods  (as a percentage of request):  0% (1m) / 50%
Min replicas:                                          1
Max replicas:                                          10
Deployment pods:                                       1 current / 1 desired
Conditions:
  Type            Status  Reason            Message
  ----            ------  ------            -------
  AbleToScale     True    ReadyForNewScale  recommended size matches current size
  ScalingActive   True    ValidMetricFound  the HPA was able to successfully calculate a replica count from cpu resource utilization (percentage of request)
  ScalingLimited  True    TooFewReplicas    the desired replica count is less than the minimum replica count
Events:           <none>
```

For this HorizontalPodAutoscaler, you can see several conditions in a healthy state. The first, `AbleToScale`, indicates whether or not the HPA is able to fetch and update scales, as well as whether or not any backoff-related conditions would prevent scaling. 

The second, `ScalingActive`, indicates whether or not the HPA is enabled (i.e. the replica count of the target is not zero) and is able to calculate desired scales. When it is False, it generally indicates problems with fetching metrics. 

Finally, the last condition, `ScalingLimited`, indicates that the desired scale was not capped by the maximum or minimum of the HorizontalPodAutoscaler. When False, it indicates that you may wish to raise or lower the minimum or maximum replica count constraints on your HorizontalPodAutoscaler.



### Quantities

All metrics in the HorizontalPodAutoscaler and metrics APIs are specified using a special whole-number notation known in Kubernetes as a **quantity**. For example, the quantity 10500m would be written as 10.5 in decimal notation. The metrics APIs will return whole numbers without a suffix when possible, and will generally return quantities in milli-units otherwise. This means you might see your metric value fluctuate between 1 and 1500m, or 1 and 1.5 when written in decimal notation.

### Other possible scenario: Creating the autoscaler declaratively

Instead of using `kubectl autoscale` command to create a HorizontalPodAutoscaler imperatively we can use the following manifest to create it declaratively:

```yaml
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: php-apache
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: php-apache
  minReplicas: 1
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 50
```

Then, create the autoscaler by executing the following command:

```sh
kubectl create -f https://k8s.io/examples/application/hpa/php-apache.yaml
```

### Figuras

<img src="prints/01-increasing_the_load.png"  style="width:50%;"/>

------------

<img src="prints/02-second_execution_and_getting_deployment.png"  style="width:50%;"/>

------------

<img src="prints/03-stop_generating_the_load.png"  style="width:50%;"/>

------------

<img src="prints/04-stop_load_deployment.png"  style="width:50%;"/>

------------

<img src="prints/05-autoscaling_v2.png"  style="width:50%;"/>

------------

<img src="prints/06-status_conditions.png"  style="width:50%;"/>

## Scale pod deployments with Horizontal Pod Autoscaler - AWS EKS

https://docs.aws.amazon.com/eks/latest/userguide/horizontal-pod-autoscaler.html

Vou fazer com os $100 do AWS Academy Learner Lab [85940]

https://awsacademy.instructure.com/courses/85940/modules/items/7812386

The Kubernetes `Horizontal Pod Autoscaler` automatically scales the number of Pods in a deployment, replication controller, or replica set based on that resource's CPU utilization. This can help your applications scale out to meet increased demand or scale in when resources are not needed, thus freeing up your nodes for other applications.

When you set a target CPU utilization percentage, the Horizontal Pod Autoscaler scales your application in or out to try to meet that target.

The Horizontal Pod Autoscaler is a standard API resource in Kubernetes that simply requires that a metrics source (such as the Kubernetes metrics server) is installed on your Amazon EKS cluster to work. You do not need to deploy or install the Horizontal Pod Autoscaler on your cluster to begin scaling your applications. 

#### Prerequisites:

- Have an existing EKS cluster: https://docs.aws.amazon.com/eks/latest/userguide/getting-started.html
- Have the Kubernetes Metrics Server instaled: https://docs.aws.amazon.com/eks/latest/userguide/metrics-server.html
- Are using a `kubectl`client that is configured to communicate with your EKS cluster: https://docs.aws.amazon.com/eks/latest/userguide/getting-started-console.html#eks-configure-kubectl